In [202]:
#This is the notebook I will use to create my logistic regression models

In [203]:
import numpy as np
import pandas as pd
%pylab inline
import csv
import statsmodels.api as sm

Populating the interactive namespace from numpy and matplotlib


In [204]:
mdiag_cols = ['CASEID','AMHTXRC3','AMHSVTYP','AMHTXND2','SPDYR','SPDMON','K6SCYR','K6SCMON','K6SCMAX','WHODASC2',
              'SMIPP_U','AMDELT','AMDEYR','ATXMDEYR','ARXMDEYR','IRINSUR4','GOVTPROG',
              'INCOME','POVERTY2','IRSEX','IRMARIT','HEALTH2','CATAG3','NEWRACE2','EDUCCAT2']
Mental_Diagnostics = pd.read_csv('C:/Users/Cody/Downloads/NSDUH/2014/NSDUH-2014-DS0001-data/NSDUH-2014-DS0001-data-excel.tsv',usecols=mdiag_cols, sep='\t', index_col=0)

In [205]:
#change to N/A for missing data
Mental_Diagnostics.AMHTXRC3.astype('int')
Mental_Diagnostics.AMHTXRC3.replace(int(2),int(0),inplace=True)#received any mental health trt past year
Mental_Diagnostics.replace(-9, np.nan, inplace=True)
Mental_Diagnostics.IRMARIT.replace(99,np.nan,inplace=True)#1-married,2-widowed,3-seperated/div,4-never married
#change vars coded 1:yes, 2:no --> 1:yes, 0:no 
Mental_Diagnostics.AMHTXND2.replace(2,0,inplace=True)#perceived need for mental health treatment in past year
Mental_Diagnostics.AMDELT.replace(2,0,inplace=True)#lifetime major depressive episode
Mental_Diagnostics.AMDEYR.replace(2,0,inplace=True)#past year major depressive episode
Mental_Diagnostics.GOVTPROG.replace(2,0,inplace=True)#participate in government assistance programs
Mental_Diagnostics.IRINSUR4.replace(2,0,inplace=True)#1-has insurance, 0-no insurance
Mental_Diagnostics.IRSEX.replace(2,0,inplace=True)#1-male, 0-female

In [206]:
mh_sample = Mental_Diagnostics[Mental_Diagnostics.SPDYR==1]
#SPDYR = 1(subsample of 5696 adults) experienced psychological distress this past year
#my logistic regression will apply to the population of adults who experienced severe psychological distress in the past year in the U.S.

In [207]:
mh_sample.columns

Index(['AMHTXRC3', 'AMHSVTYP', 'AMHTXND2', 'K6SCMON', 'SPDMON', 'K6SCYR',
       'K6SCMAX', 'SPDYR', 'WHODASC2', 'SMIPP_U', 'AMDELT', 'AMDEYR',
       'ATXMDEYR', 'ARXMDEYR', 'GOVTPROG', 'INCOME', 'POVERTY2', 'IRINSUR4',
       'IRSEX', 'IRMARIT', 'CATAG3', 'NEWRACE2', 'EDUCCAT2', 'HEALTH2'],
      dtype='object')

In [208]:
dummy_ages = pd.get_dummies(mh_sample['CATAG3'],prefix='age') # 2:18-25, 3:26-34, 4:35-49, 5:50+
dummy_marriage = pd.get_dummies(mh_sample['IRMARIT'],prefix='marriage') # 1-married, 2-widowed, 3-sep/div, 4-unmarried
dummy_race = pd.get_dummies(mh_sample['NEWRACE2'],prefix='race') # 1-White, 2-Black, 3-Native American, 4-Pacific Islander, 5-Asian, 6-Mixed, 7-Hispanic
dummy_health = pd.get_dummies(mh_sample['HEALTH2'],prefix='health') # overall health(1-excellent, 2-very good, 3-good, 4-fair/poor)
dummy_education = pd.get_dummies(mh_sample['EDUCCAT2'],prefix='education') #1-less than high school, 2-high school grad, 3-some college, 4-college graduate

In [231]:
#create a table with just the columns I want to use for logistic regression
cols_to_keep = ['AMHTXRC3','AMHTXND2','WHODASC2','SMIPP_U','AMDELT','IRSEX']
data = mh_sample[cols_to_keep].copy()
data = data.join(dummy_ages.ix[:,'age_3':])
data = data.join(dummy_race.ix[:,'race_2':])
data = data.join(dummy_health.ix[:,'health_2':])
#data = data.join(dummy_education.ix[:,'education_2':])
#data = data.join(dummy_marriage.ix[:,'marriage_2':])
data['intercept'] = 1
#finally realized that it was the n/a values in the data messing up my logistic regression, so I dropped the rows with N/A values
data = data[pd.notnull(data['AMHTXRC3'])]
data = data[pd.notnull(data['AMHTXND2'])]
data = data[pd.notnull(data['AMDELT'])]
data.describe()

,AMHTXRC3,AMHTXND2,WHODASC2,SMIPP_U,AMDELT,IRSEX,age_3,age_4,age_5,race_2,race_3,race_4,race_5,race_6,race_7,health_2.0,health_3.0,health_4.0,intercept
count,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.0
mean,0.445332,0.294462,11.972666,0.227417,0.544551,0.349840,0.212460,0.212638,0.114306,0.102946,0.016862,0.005325,0.036209,0.044373,0.148740,0.337771,0.308129,0.220447,1.0
std,0.497047,0.455841,6.245606,0.260468,0.498055,0.476962,0.409085,0.409210,0.318211,0.303916,0.128765,0.072783,0.186826,0.205942,0.355863,0.472992,0.461761,0.414585,0.0
min,0.000000,0.000000,0.000000,0.004284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.000000,8.000000,0.029295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,0.000000,0.000000,12.000000,0.106492,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.000000,1.000000,16.000000,0.355731,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.0
max,1.000000,1.000000,24.000000,0.929121,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [232]:
#now I have my table with all of the potential predictor variables
#I am looking into a method that will select the K best predictors for my response variable AMHTXRC3

In [233]:
train_cols = data.columns[1:]
#train_cols = data['AMHTXRC3','AMHTXND2','WHODASC2','SMIPP_U','AMDELT','IRSEX','age_2','race_1','health_2.0']


In [234]:
logit = sm.Logit(data['AMHTXRC3'],data[train_cols])
result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.578526
         Iterations 6


In [235]:
result.summary()
#I tried to fit a model with all of the variables and the variables with the highest p-values in order are:
#education_2, marriage_3, marriage_4, marriage_2, education_3, education_4, K6SCMAX eliminated
#race_3, race_4, race_6, health_2, health_3
#I will use backwards variable selection to reduce the model to something more realistic and useful
#Update: Reduced to 8 variables(including categoricals)

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               AMHTXRC3   No. Observations:                 5634
Model:                          Logit   Df Residuals:                     5616
Method:                           MLE   Df Model:                           17
Date:                Sun, 14 Aug 2016   Pseudo R-squ.:                  0.1581
Time:                        20:58:27   Log-Likelihood:                -3259.4
converged:                       True   LL-Null:                       -3871.4
                                        LLR p-value:                9.043e-250
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
AMHTXND2       0.3578      0.069      5.211      0.000         0.223     0.492
WHODASC2       0.0591      0.007      8.911      0.000         0.046     0.072
SMIPP_U        0.9599      0.172      5.592      0.000         0.623     1.296
AMDELT         0.3851      0.069      5.586      0.000         0.250     0.520
IRSEX         -0.3863      0.064     -6.032      0.000        -0.512    -0.261
age_3          0.4507      0.078      5.752      0.000         0.297     0.604
age_4          0.7440      0.080      9.255      0.000         0.586     0.902
age_5          0.9998      0.104      9.587      0.000         0.795     1.204
race_2        -0.7687      0.106     -7.279      0.000        -0.976    -0.562
race_3        -0.6712      0.241     -2.782      0.005        -1.144    -0.198
race_4        -1.1125      0.461     -2.411      0.016        -2.017    -0.208
race_5        -1.0827      0.188     -5.772      0.000        -1.450    -0.715
race_6        -0.4333      0.148     -2.929      0.003        -0.723    -0.143
race_7        -0.7384      0.090     -8.199      0.000        -0.915    -0.562
health_2.0     0.3068      0.100      3.068      0.002         0.111     0.503
health_3.0     0.3032      0.101      2.990      0.003         0.104     0.502
health_4.0     0.4805      0.111      4.339      0.000         0.263     0.698
intercept     -1.7800      0.119    -14.958      0.000        -2.013    -1.547
==============================================================================
"""

In [236]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%','97.5%','OR']
np.exp(conf)
#the table below shows us the 95% confidence interval for the odds ratios of each predictor
#this is the odds ratio for receiving treatment for those with serious psychological distress in the past year

,2.5%,97.5%,OR
AMHTXND2,1.250068,1.636082,1.430109
WHODASC2,1.047220,1.074825,1.060933
SMIPP_U,1.865406,3.655730,2.611402
AMDELT,1.284033,1.682501,1.469825
IRSEX,0.599425,0.770473,0.679589
age_3,1.346024,1.830044,1.569485
age_4,1.797583,2.463455,2.104344
age_5,2.215329,3.334002,2.717703
race_2,0.376956,0.570239,0.463632
race_3,0.318509,0.820049,0.511071


In [243]:
cols_to_keep = ['AMHTXRC3','WHODASC2','AMDELT','IRSEX']
data = mh_sample[cols_to_keep].copy()
data = data.join(dummy_ages.ix[:,'age_3':])
data = data.join(dummy_race.ix[:,'race_2':])
data = data.join(dummy_health.ix[:,'health_2':])
data['intercept'] = 1
data = data[pd.notnull(data['AMHTXRC3'])]
data = data[pd.notnull(data['AMDELT'])]

In [244]:
train_cols = data.columns[1:]

In [245]:
logit = sm.Logit(data['AMHTXRC3'],data[train_cols])
result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.585997
         Iterations 6


In [246]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               AMHTXRC3   No. Observations:                 5646
Model:                          Logit   Df Residuals:                     5630
Method:                           MLE   Df Model:                           15
Date:                Sun, 14 Aug 2016   Pseudo R-squ.:                  0.1472
Time:                        21:11:14   Log-Likelihood:                -3308.5
converged:                       True   LL-Null:                       -3879.8
                                        LLR p-value:                3.474e-234
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
WHODASC2       0.0856      0.005     15.749      0.000         0.075     0.096
AMDELT         0.5546      0.064      8.668      0.000         0.429     0.680
IRSEX         -0.3835      0.063     -6.073      0.000        -0.507    -0.260
age_3          0.4816      0.077      6.236      0.000         0.330     0.633
age_4          0.7673      0.079      9.716      0.000         0.613     0.922
age_5          0.9876      0.103      9.614      0.000         0.786     1.189
race_2        -0.7433      0.104     -7.125      0.000        -0.948    -0.539
race_3        -0.6614      0.240     -2.759      0.006        -1.131    -0.192
race_4        -1.2086      0.461     -2.621      0.009        -2.113    -0.305
race_5        -1.0788      0.185     -5.841      0.000        -1.441    -0.717
race_6        -0.4105      0.146     -2.810      0.005        -0.697    -0.124
race_7        -0.7462      0.089     -8.370      0.000        -0.921    -0.571
health_2.0     0.3354      0.099      3.382      0.001         0.141     0.530
health_3.0     0.3461      0.101      3.443      0.001         0.149     0.543
health_4.0     0.5796      0.109      5.299      0.000         0.365     0.794
intercept     -1.9307      0.115    -16.852      0.000        -2.155    -1.706
==============================================================================
"""

In [247]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%','97.5%','OR']
np.exp(conf)
#the table below shows us the 95% confidence interval for the odds ratios of each predictor
#this is the odds ratio for receiving treatment for those with serious psychological distress in the past year

,2.5%,97.5%,OR
WHODASC2,1.077792,1.100992,1.089330
AMDELT,1.536030,1.973886,1.741249
IRSEX,0.602143,0.771250,0.681471
age_3,1.391297,1.883183,1.618662
age_4,1.845142,2.514626,2.154029
age_5,2.195182,3.283601,2.684791
race_2,0.387600,0.583417,0.475533
race_3,0.322619,0.825659,0.516114
race_4,0.120930,0.737333,0.298606
race_5,0.236753,0.488316,0.340015
